## Chapter 2

#### BA2A  | Implement MotifEnumeration

In [1]:
def HammingDist(string1, string2):
    dist = 0
    for i, nt in enumerate(string1):
        if nt != string2[i]:
            dist += 1
    return dist

In [2]:
def Neighbors(pattern, d):
    if d == 0:
        return {pattern}
    nucleotides = {'A','C','G','T'}
    if len(pattern) == 1:
        return nucleotides
    neighborhood = set()
    suffix_neighbors = Neighbors(pattern[1:],d)
    for text in suffix_neighbors:
        if HammingDist(pattern[1:],text) < d:
            for base in nucleotides:
                neighborhood.add(base+text)
        else:
            neighborhood.add(pattern[0]+text)

    return neighborhood

In [3]:
# get Neighborhood for each dna, and intersect all neighborhoods
def MotifEnumeration(k, d, dnas):
    neighbor_list = []
    for dna in dnas:
        neighbor = set()
        for i in range(len(dna)-k+1):
            neighbor = neighbor.union(Neighbors(dna[i:i+k], d))
        neighbor_list.append(neighbor)
    motifs = set.intersection(*neighbor_list)
    return motifs

In [4]:
with open("../data/rosalind_ba2a.txt") as input_file:
    k, d = map(int, input_file.readline().split())
    dna_list = [line.strip() for line in input_file.readlines()]

for motif in MotifEnumeration(k, d, dna_list):
    print motif,

ATT TTT GTT ATA


### BA2B | Find a Median String

In [5]:
def HammingDist(string1, string2):
    dist = 0
    for i, nt in enumerate(string1):
        if nt != string2[i]:
            dist += 1
    return dist

In [6]:
def NumToPattern(integer,k):
    num_str = ""
    for i in range(k):
        remainder = (integer/(4**(k-i-1)))%4
        if remainder == 0:
            num_str += "A"
        elif remainder == 1:
            num_str += "C"
        elif remainder == 2:
            num_str += "G"
        elif remainder == 3:
            num_str += "T"
    return num_str

In [7]:
def DistBtwPatternAndStrings(pattern, dnas):
    k = len(pattern)
    dist = 0
    for dna in dnas:
        hamming_dist = float('inf')
        for i in range(len(dna)-k+1):
            hd = HammingDist(pattern, dna[i:i+k])
            if hamming_dist > hd:
                hamming_dist = hd
        dist += hamming_dist
    return dist

In [10]:
def MedianString(k, dnas):
    dist = float('inf')
    median = ''
    for i in range(4**k):
        pattern = NumToPattern(i,k)
        d = DistBtwPatternAndStrings(pattern, dnas)
        if dist > d:
            dist = d
            median = pattern
    return median

In [11]:
with open("../data/rosalind_ba2b.txt") as input_file:
    k_val = int(input_file.readline())
    dna_list = [line.strip() for line in input_file.readlines()]

In [12]:
MedianString(k_val, dna_list)

'TCCGTA'

### BA2C | Find a Profile-most Probable k-mer in a String

In [13]:
def ProfileMostProbableKmer(dna, k, profile):
    max_kmer_pair = [-1, None]
    for i in range(len(dna)-k+1):
        kmer = dna[i:i+k]
        prob = 1
        for j in range(len(kmer)):
            if kmer[j] == "A":
                prob *= float(profile[0][j])
            elif kmer[j] == "C":
                prob *= float(profile[1][j])
            elif kmer[j] == "G":
                prob *= float(profile[2][j])
            elif kmer[j] == "T":
                prob *= float(profile[3][j])
        if prob > max_kmer_pair[0]:
            max_kmer_pair = [prob, kmer]
    return max_kmer_pair[1]

In [16]:
with open("../data/rosalind_ba2c.txt") as input_file:
    dna = input_file.readline().rstrip('\n')
    k = int(input_file.readline())
    profile = [row.strip().split(' ') for row in input_file]

In [17]:
ProfileMostProbableKmer(dna, k, profile)

'GACGCGC'

### BA2D | Implement GreedyMotifSearch

In [18]:
def HammingDist(string1, string2):
    dist = 0
    for i, nt in enumerate(string1):
        if nt != string2[i]:
            dist += 1
    return dist

In [19]:
def BuildProfile(motifs):
    profile  = []
    for i in range(len(motifs[0])):
        col = ''.join([row[i] for row in motifs])
        profile.append([float(col.count(nt))/float(len(motifs)) for nt in 'ACGT'])
    return [list(i) for i in zip(*profile)]

In [20]:
def Score(motifs):
    score = 0
    for i in range(len(motifs[0])):
        col = ''.join([row[i] for row in motifs]) 
        score += min([HammingDist(col, uni_col*len(motifs)) for uni_col in 'ACGT'])
    return score

In [21]:
def GreedyMotifSearch(dnas, k, t):
    best_motifs = [dna[0:k] for dna in dnas]
    for i in range(len(dna[0])-k+1):
        motifs = [dna[0][i:i+k]]
        profile = BuildProfile(motifs)
        for j in range(1,t):
            motifs.append(ProfileMostProbableKmer(dna[j], k, profile))
            profile = BuildProfile(motifs)
            
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
    return best_motifs

In [22]:
with open('../data/rosalind_ba2d.txt') as input_file:
    k,t = map(int, input_file.readline().split())
    dnas = [line.strip() for line in input_file.readlines()]

In [23]:
for dx in GreedyMotifSearch(dnas, k, t):
    print dx

GGC
AAG
CAA
CAC
CAA


### BA2E | Implement GreedyMotifSearch with Pseudocounts

In [24]:
def BuildProfilePseudocounts(motifs):
    profile  = []
    for i in range(len(motifs[0])):
        col = ''.join([row[i] for row in motifs])
        profile.append([(float(col.count(nt))+1)/float(len(motifs)) for nt in 'ACGT'])
    return [list(i) for i in zip(*profile)]

In [25]:
def Score(motifs):
    score = 0
    for i in range(len(motifs[0])):
        col = ''.join([row[i] for row in motifs]) 
        score += min([hamming_dist(col, uni_col*len(motifs)) for uni_col in 'ACGT'])
    return score

In [26]:
def GreedyMotifSearchPseudoCounts(dnas, k, t):
    best_motifs = [dna[0:k] for dna in dnas]
    for i in range(len(dna[0])-k+1):
        motifs = [dna[0][i:i+k]]
        profile = BuildProfilePseudocounts(motifs)
        for j in range(1,t):
            motifs.append(ProfileMostProbableKmer(dna[j], k, profile))
            profile = BuildProfilePseudocounts(motifs)
            
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
    return best_motifs

In [28]:
with open('../data/rosalind_ba2e.txt') as input_file:
    k,t = map(int, input_file.readline().split())
    dnas = [line.strip() for line in input_file.readlines()]

In [29]:
GreedyMotifSearchPseudoCounts(dnas, k, t)

['GGC', 'AAG', 'CAA', 'CAC', 'CAA']

### BA2F | Implement RandomizedMotifSearch

In [30]:
def HammingDist(string1, string2):
    dist = 0
    for i, nt in enumerate(string1):
        if nt != string2[i]:
            dist += 1
    return dist

In [31]:
def BuildProfilePseudocounts(motifs):
    profile  = []
    for i in range(len(motifs[0])):
        col = ''.join([row[i] for row in motifs])
        profile.append([(float(col.count(nt))+1)/float(len(motifs)) for nt in 'ACGT'])
    return [list(i) for i in zip(*profile)]

In [32]:
def Score(motifs):
    score = 0
    for i in range(len(motifs[0])):
        col = ''.join([row[i] for row in motifs]) 
        score += min([HammingDist(col, uni_col*len(motifs)) for uni_col in 'ACGT'])
    return score

In [33]:
def MotifsGivenProfile(profile, dnas, k):
    motifs =[]
    for dna in dnas:
        motifs.append(ProfileMostProbableKmer(dna,k,profile))
    return motifs    

In [34]:
def ProfileMostProbableKmer(dna, k, profile):
    max_kmer_pair = [-1, None]
    for i in range(len(dna)-k+1):
        kmer = dna[i:i+k]
        prob = 1
        for j in range(len(kmer)):
            if kmer[j] == "A":
                prob *= float(profile[0][j])
            elif kmer[j] == "C":
                prob *= float(profile[1][j])
            elif kmer[j] == "G":
                prob *= float(profile[2][j])
            elif kmer[j] == "T":
                prob *= float(profile[3][j])
        if prob > max_kmer_pair[0]:
            max_kmer_pair = [prob, kmer]
    return max_kmer_pair[1]

In [35]:
from random import randint
def RandMotifSearch(dnas, k, t):
    # Generalize random motifs
    rand_motifs = []
    for dna in dnas:
        rand_int = randint(0,len(dna)-k)
        rand_motifs.append(dna[rand_int:rand_int+k])
    best_motifs = rand_motifs
    while True:
        profile = BuildProfilePseudocounts(best_motifs)
        motifs = MotifsGivenProfile(profile,dnas,k)

        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
        else:
            return best_motifs        

In [36]:
def RepeatRandMotifSearch(dnas,k,t):
    best_score = float('inf')
    best_motifs =[]
    i = 0
    while i <1000 :
        motifs = RandMotifSearch(dnas,k,t)

        if Score(motifs) < best_score:
            best_score = Score(motifs)
            best_motifs = motifs
        else:
            i+= 1
    return best_motifs        

In [37]:
with open('../data/rosalind_ba2f.txt') as input_file:
    k,t = map(int, input_file.readline().split())
    dnas = [line.strip() for line in input_file.readlines()]

In [38]:
res = RepeatRandMotifSearch(dnas, k, t)

In [39]:
for t in res:
    print t

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


### BA2H | Implement DistanceBetweenPatternAndStrings

In [40]:
def HammingDist(string1, string2):
    dist = 0
    for i, nt in enumerate(string1):
        if nt != string2[i]:
            dist += 1
    return dist

In [41]:
def DistBtwPatternAndStrings(pattern, dnas):
    k = len(pattern)
    dist = 0
    for dna in dnas:
        hamming_dist = float('inf')
        for i in range(len(dna)-k+1):
            hd = HammingDist(pattern, dna[i:i+k])
            if hamming_dist > hd:
                hamming_dist = hd
        dist += hamming_dist
    return dist

In [42]:
with open("../data/rosalind_ba2h.txt") as input_file:
    pattern = input_file.readline().strip()
    dna_list = input_file.readline().strip().split(' ')

In [43]:
DistBtwPatternAndStrings(pattern, dna_list)

5